In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Define custom InceptionV3 model
def custom_inceptionv3(input_shape=(224, 224, 3), num_classes=2):
    input_tensor = layers.Input(shape=input_shape)

    # Stem
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='valid', activation='relu')(input_tensor)
    x = layers.Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = layers.Conv2D(80, (1, 1), padding='valid', activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), padding='valid', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Inception blocks
    x = inception_block(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_block(x, filters=[128, 128, 192, 32, 96, 64])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[192, 96, 208, 16, 48, 64])
    x = inception_block(x, filters=[160, 112, 224, 24, 64, 64])
    x = inception_block(x, filters=[128, 128, 256, 24, 64, 64])
    x = inception_block(x, filters=[112, 144, 288, 32, 64, 64])
    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = inception_block(x, filters=[384, 192, 384, 48, 128, 128])

    # Global average pooling and output layer
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(input_tensor, output, name='custom_inceptionv3')
    return model

# Define Inception block
def inception_block(x, filters):
    branch1x1 = layers.Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    branch3x3 = layers.Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    branch3x3 = layers.Conv2D(filters[2], (3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = layers.Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    branch5x5 = layers.Conv2D(filters[4], (5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = layers.Conv2D(filters[5], (1, 1), padding='same', activation='relu')(branch_pool)

    return layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)

# Instantiate the custom InceptionV3 model
model = custom_inceptionv3()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
# model.summary()

# Paths to your train and test directories
train_dir = '/content/drive/MyDrive/2Classes/train'
test_dir = '/content/drive/MyDrive/2Classes/test'

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 96 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [8]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator))

Epoch 1/10
3/3 [==============================] - 44s 10s/step - loss: 0.6930 - accuracy: 0.4896 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 30s 10s/step - loss: 0.6962 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 31s 10s/step - loss: 0.6935 - accuracy: 0.4896 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 4/10
3/3 [==============================] - 31s 10s/step - loss: 0.6947 - accuracy: 0.4688 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 30s 10s/step - loss: 0.6940 - accuracy: 0.5208 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 30s 10s/step - loss: 0.6939 - accuracy: 0.4479 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 30s 10s/step - loss: 0.6928 - accuracy: 0.4688 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/10
3/3 [===========

In [9]:
# Extract training and validation accuracy from history
training_accuracy = (history.history['accuracy'][-1] * 100)
validation_accuracy = (history.history['val_accuracy'][-1] * 100)

print(f"Training Accuracy: {training_accuracy}")
print(f"Validation Accuracy: {validation_accuracy}")

Training Accuracy: 53.125
Validation Accuracy: 50.0
